## Restaurant Recommendation System

This Restaurant Recommendation System is used to analyze the available Restaurant data in a perticular city and adivice the user 
with the place details where he can start the new Restaurant. 

#### Importing necessary Libraries

In [2]:
import requests
import pandas as pd
import numpy as np
import random

from geopy.geocoders import Nominatim

from IPython.display import Image
from IPython.core.display import HTML

from pandas.io.json import json_normalize

!conda install -c conda-forge folium=0.5.0 --yes
import folium 

print('Folium installed')
print('Libraries imported')

Solving environment: done

## Package Plan ##

  environment location: /opt/conda/envs/Python36

  added / updated specs: 
    - folium=0.5.0


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    openssl-1.1.1g             |       h516909a_0         2.1 MB  conda-forge
    branca-0.4.1               |             py_0          26 KB  conda-forge
    vincent-0.4.4              |             py_1          28 KB  conda-forge
    altair-4.1.0               |             py_1         614 KB  conda-forge
    ca-certificates-2020.6.20  |       hecda079_0         145 KB  conda-forge
    certifi-2020.6.20          |   py36h9f0ad1d_0         151 KB  conda-forge
    folium-0.5.0               |             py_0          45 KB  conda-forge
    python_abi-3.6             |          1_cp36m           4 KB  conda-forge
    ------------------------------------------------------------
                       

## Define Foursquare credentials 

In [3]:
CLIENT_ID = 'KK2CAPVUQYDEWVDF0LXWA2H5D25OPJTZBIEY1RTQ1MZHGFF4' # your Foursquare ID
CLIENT_SECRET = 'JRVYNHJ3BZSDGQ2DMT04Z3PEQCRKE5EIV0WYQRY1OXGZUVBK' # your Foursquare Secret
VERSION = '20180604'
LIMIT = 30
print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: KK2CAPVUQYDEWVDF0LXWA2H5D25OPJTZBIEY1RTQ1MZHGFF4
CLIENT_SECRET:JRVYNHJ3BZSDGQ2DMT04Z3PEQCRKE5EIV0WYQRY1OXGZUVBK


Lets plan that we are planning to open a Chinese restaurant. For that first we find what are the chinese restaurants available in Dallas.

In [7]:
address = '3011 Gulden Ln Ste 110,Dallas,TX'

geolocator = Nominatim(user_agent="foursquare_agent")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print(latitude, longitude)

32.7792282 -96.8278896


## Search for Chinese restaurants

In [30]:
search_query = 'Chinese'
radius = 10000
print(search_query + '... Ok!')

Chinese... Ok!


Define the corresponding URL

In [31]:
url = 'https://api.foursquare.com/v2/venues/search?client_id={}&client_secret={}&ll={},{}&v={}&query={}&radius={}&limit={}'.format(CLIENT_ID, CLIENT_SECRET, latitude, longitude, VERSION, search_query, radius, LIMIT)
url

'https://api.foursquare.com/v2/venues/search?client_id=KK2CAPVUQYDEWVDF0LXWA2H5D25OPJTZBIEY1RTQ1MZHGFF4&client_secret=JRVYNHJ3BZSDGQ2DMT04Z3PEQCRKE5EIV0WYQRY1OXGZUVBK&ll=32.7792282,-96.8278896&v=20180604&query=Chinese&radius=10000&limit=30'

In [32]:
results = requests.get(url).json()
results

{'meta': {'code': 200, 'requestId': '5f13e2633c1ce60ee8392df9'},
 'response': {'venues': [{'id': '5ee910fed7054d000853856f',
    'name': 'Sum Dang Good Chinese',
    'location': {'address': '3011 Gulden Ln',
     'lat': 32.779453,
     'lng': -96.828723,
     'labeledLatLngs': [{'label': 'display',
       'lat': 32.779453,
       'lng': -96.828723}],
     'distance': 81,
     'postalCode': '75212',
     'cc': 'US',
     'city': 'Dallas',
     'state': 'TX',
     'country': 'United States',
     'formattedAddress': ['3011 Gulden Ln',
      'Dallas, TX 75212',
      'United States']},
    'categories': [{'id': '4bf58dd8d48988d145941735',
      'name': 'Chinese Restaurant',
      'pluralName': 'Chinese Restaurants',
      'shortName': 'Chinese',
      'icon': {'prefix': 'https://ss3.4sqi.net/img/categories_v2/food/asian_',
       'suffix': '.png'},
      'primary': True}],
    'delivery': {'id': '2048307',
     'url': 'https://www.grubhub.com/restaurant/sum-dang-good-3011-gulden-ln-dallas

In [33]:
venues = results['response']['venues']

dataframe = json_normalize(venues)
dataframe.head()

,categories,delivery.id,delivery.provider.icon.name,delivery.provider.icon.prefix,delivery.provider.icon.sizes,delivery.provider.name,delivery.url,hasPerk,id,location.address,...,location.crossStreet,location.distance,location.formattedAddress,location.labeledLatLngs,location.lat,location.lng,location.postalCode,location.state,name,referralId
0,"[{'id': '4bf58dd8d48988d145941735', 'name': 'C...",2048307,/delivery_provider_grubhub_20180129.png,https://fastly.4sqi.net/img/general/cap/,"[40, 50]",grubhub,https://www.grubhub.com/restaurant/sum-dang-go...,False,5ee910fed7054d000853856f,3011 Gulden Ln,...,NaN,81,"[3011 Gulden Ln, Dallas, TX 75212, United States]","[{'label': 'display', 'lat': 32.779453, 'lng':...",32.779453,-96.828723,75212,TX,Sum Dang Good Chinese,v-1595138907
1,"[{'id': '52af3b773cf9994f4e043c03', 'name': 'S...",545184,/delivery_provider_grubhub_20180129.png,https://fastly.4sqi.net/img/general/cap/,"[40, 50]",grubhub,https://www.grubhub.com/restaurant/szechuan-ch...,False,55e64275498e7263eeda1593,4411 Lemmon Ave,...,NaN,4578,"[4411 Lemmon Ave, Dallas, TX 75219, United Sta...","[{'label': 'display', 'lat': 32.8185180017725,...",32.818518,-96.813432,75219,TX,Szechuan Chinese Restaurant,v-1595138907
2,"[{'id': '4bf58dd8d48988d145941735', 'name': 'C...",1876400,/delivery_provider_grubhub_20180129.png,https://fastly.4sqi.net/img/general/cap/,"[40, 50]",grubhub,https://www.grubhub.com/restaurant/valentino-c...,False,4c2be808d1a10f47bcc0f864,526 E Jefferson Blvd,...,NaN,3837,"[526 E Jefferson Blvd, Dallas, TX 75203, Unite...","[{'label': 'display', 'lat': 32.74575554018678...",32.745756,-96.818083,75203,TX,Valentino Chinese Restuarant,v-1595138907
3,"[{'id': '4bf58dd8d48988d145941735', 'name': 'C...",NaN,NaN,NaN,NaN,NaN,NaN,False,4a7e1d70f964a52005f11fe3,3607 Greenville Ave,...,at Martel Av,7906,"[3607 Greenville Ave (at Martel Av), Dallas, T...","[{'label': 'display', 'lat': 32.83123007847272...",32.831230,-96.770337,75206,TX,Betty's Best Around Chinese Restaurant,v-1595138907
4,"[{'id': '4bf58dd8d48988d145941735', 'name': 'C...",NaN,NaN,NaN,NaN,NaN,NaN,False,4e5d2025a8092f6396805fd4,1710 E State Highway 356,...,NaN,9594,"[1710 E State Highway 356, Irving, TX 75060, U...","[{'label': 'display', 'lat': 32.80948334084759...",32.809483,-96.923898,75060,TX,Subs-A Salads & Chinese Buffet,v-1595138907


In [34]:
filtered_columns = ['name', 'categories'] + [col for col in dataframe.columns if col.startswith('location.')] + ['id']
dataframe_filtered = dataframe.loc[:, filtered_columns]

# function that extracts the category of the venue
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

# filter the category for each row
dataframe_filtered['categories'] = dataframe_filtered.apply(get_category_type, axis=1)

# clean column names by keeping only last term
dataframe_filtered.columns = [column.split('.')[-1] for column in dataframe_filtered.columns]

dataframe_filtered

,name,categories,address,cc,city,country,crossStreet,distance,formattedAddress,labeledLatLngs,lat,lng,postalCode,state,id
0,Sum Dang Good Chinese,Chinese Restaurant,3011 Gulden Ln,US,Dallas,United States,NaN,81,"[3011 Gulden Ln, Dallas, TX 75212, United States]","[{'label': 'display', 'lat': 32.779453, 'lng':...",32.779453,-96.828723,75212,TX,5ee910fed7054d000853856f
1,Szechuan Chinese Restaurant,Szechuan Restaurant,4411 Lemmon Ave,US,Dallas,United States,NaN,4578,"[4411 Lemmon Ave, Dallas, TX 75219, United Sta...","[{'label': 'display', 'lat': 32.8185180017725,...",32.818518,-96.813432,75219,TX,55e64275498e7263eeda1593
2,Valentino Chinese Restuarant,Chinese Restaurant,526 E Jefferson Blvd,US,Dallas,United States,NaN,3837,"[526 E Jefferson Blvd, Dallas, TX 75203, Unite...","[{'label': 'display', 'lat': 32.74575554018678...",32.745756,-96.818083,75203,TX,4c2be808d1a10f47bcc0f864
3,Betty's Best Around Chinese Restaurant,Chinese Restaurant,3607 Greenville Ave,US,Dallas,United States,at Martel Av,7906,"[3607 Greenville Ave (at Martel Av), Dallas, T...","[{'label': 'display', 'lat': 32.83123007847272...",32.831230,-96.770337,75206,TX,4a7e1d70f964a52005f11fe3
4,Subs-A Salads & Chinese Buffet,Chinese Restaurant,1710 E State Highway 356,US,Irving,United States,NaN,9594,"[1710 E State Highway 356, Irving, TX 75060, U...","[{'label': 'display', 'lat': 32.80948334084759...",32.809483,-96.923898,75060,TX,4e5d2025a8092f6396805fd4
5,Chen's chinese kitchen,Chinese Restaurant,Pearl st,US,Dallas,United States,NaN,2958,"[Pearl st, Dallas, TX, United States]","[{'label': 'display', 'lat': 32.78657798160772...",32.786578,-96.797516,NaN,TX,4c755ce8db52b1f7c68678dc
6,Dragon Palace Chinese Restaurant,Chinese Restaurant,Village Plaza,US,Dallas,United States,Constant Spring Road,2520,"[Village Plaza (Constant Spring Road), Dallas,...","[{'label': 'display', 'lat': 32.78005925294478...",32.780059,-96.800979,75202,TX,5114228ce4b02cf782f651b4
7,Panda's Restaurant & Bar,Chinese Restaurant,3917 Cedar Springs Rd,US,Dallas,United States,at Throckmorton St.,3848,"[3917 Cedar Springs Rd (at Throckmorton St.), ...","[{'label': 'display', 'lat': 32.81077020211321...",32.810770,-96.811049,75219,TX,4b1a8a2af964a520e0eb23e3
8,Chino's Chinese Restaurant,Chinese Restaurant,1050 N Westmoreland Rd,US,Dallas,United States,NaN,4937,"[1050 N Westmoreland Rd, Dallas, TX 75211, Uni...","[{'label': 'display', 'lat': 32.75555419921875...",32.755554,-96.872498,75211,TX,4e5d2057b61cc7d9384414d7
9,chinese kitchen oc,Asian Restaurant,NaN,US,NaN,United States,NaN,9271,"[Texas, United States]","[{'label': 'display', 'lat': 32.69676576987252...",32.696766,-96.841766,NaN,Texas,4e6e96bb7d8b6c5625c4700f


In [35]:
dataframe_filtered.name

0                        Sum Dang Good Chinese
1                  Szechuan Chinese Restaurant
2                 Valentino Chinese Restuarant
3       Betty's Best Around Chinese Restaurant
4               Subs-A Salads & Chinese Buffet
5                       Chen's chinese kitchen
6             Dragon Palace Chinese Restaurant
7                     Panda's Restaurant & Bar
8                   Chino's Chinese Restaurant
9                           chinese kitchen oc
10                        pans chinese buffett
11                       Chin Big Wong Chinese
12                        Promise Chinese Food
13                             Chinese Kitchen
14             Dragon Pearl Chinese Restaurant
15                       The Wall Chinese Cafe
16                             Wah Wah Chinese
17                             Wah Wah Chinese
18            Liu's Kitchen Chinese Restaurant
19             Royal Chopstick Chinese Express
20    Jia Modern Chinese Dallas & Asian Lounge
21           

In [36]:
venues_map = folium.Map(location=[latitude, longitude], zoom_start=13) # generate map centred around the Conrad Hotel

# add a red circle marker to represent the Conrad Hotel
folium.features.CircleMarker(
    [latitude, longitude],
    radius=10,
    color='red',
    popup='Conrad Hotel',
    fill = True,
    fill_color = 'red',
    fill_opacity = 0.6
).add_to(venues_map)

# add the Italian restaurants as blue circle markers
for lat, lng, label in zip(dataframe_filtered.lat, dataframe_filtered.lng, dataframe_filtered.categories):
    folium.features.CircleMarker(
        [lat, lng],
        radius=5,
        color='blue',
        popup=label,
        fill = True,
        fill_color='blue',
        fill_opacity=0.6
    ).add_to(venues_map)

# display map
venues_map

## Explore the given Venue

In [15]:
venue_id = '5ee910fed7054d000853856f'
url = 'https://api.foursquare.com/v2/venues/{}?client_id={}&client_secret={}&v={}'.format(venue_id, CLIENT_ID, CLIENT_SECRET, VERSION)
url

'https://api.foursquare.com/v2/venues/5ee910fed7054d000853856f?client_id=KK2CAPVUQYDEWVDF0LXWA2H5D25OPJTZBIEY1RTQ1MZHGFF4&client_secret=JRVYNHJ3BZSDGQ2DMT04Z3PEQCRKE5EIV0WYQRY1OXGZUVBK&v=20180604'

In [16]:
result = requests.get(url).json()
print(result['response']['venue'].keys())
result['response']['venue']

dict_keys(['id', 'name', 'contact', 'location', 'canonicalUrl', 'categories', 'verified', 'stats', 'price', 'likes', 'dislike', 'ok', 'delivery', 'allowMenuUrlEdit', 'beenHere', 'specials', 'photos', 'reasons', 'hereNow', 'createdAt', 'tips', 'shortUrl', 'timeZone', 'listed', 'seasonalHours', 'pageUpdates', 'inbox', 'parent', 'hierarchy', 'attributes'])


{'id': '5ee910fed7054d000853856f',
 'name': 'Sum Dang Good Chinese',
 'contact': {},
 'location': {'address': '3011 Gulden Ln',
  'lat': 32.779453,
  'lng': -96.828723,
  'labeledLatLngs': [{'label': 'display',
    'lat': 32.779453,
    'lng': -96.828723}],
  'postalCode': '75212',
  'cc': 'US',
  'city': 'Dallas',
  'state': 'TX',
  'country': 'United States',
  'formattedAddress': ['3011 Gulden Ln', 'Dallas, TX 75212', 'United States']},
 'canonicalUrl': 'https://foursquare.com/v/sum-dang-good-chinese/5ee910fed7054d000853856f',
 'categories': [{'id': '4bf58dd8d48988d145941735',
   'name': 'Chinese Restaurant',
   'pluralName': 'Chinese Restaurants',
   'shortName': 'Chinese',
   'icon': {'prefix': 'https://ss3.4sqi.net/img/categories_v2/food/asian_',
    'suffix': '.png'},
   'primary': True}],
 'verified': False,
 'stats': {'tipCount': 0},
 'price': {'tier': 1, 'message': 'Cheap', 'currency': '$'},
 'likes': {'count': 1,
  'groups': [{'type': 'others', 'count': 1, 'items': []}],
  '

In [18]:
result['response']['venue']['tips']['count']

0